In [ ]:
# Google.colab drive integration
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# specify the desired data location to the current working directory
%cd /content/drive/MyDrive/Dataset

/content/drive/.shortcut-targets-by-id/1hQJhZXCBbc369GWFH06FCrflHQ8EO2xS/Dataset


In [ ]:
# Keras prepocessing library
! pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 711.8 kB/s eta 0:00:00


In [ ]:
# Get the required Library
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50V2, VGG16

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, MaxPool2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [ ]:
batch_size = 32

In [ ]:
# Training Data Path and Image gen (Normalization)
TRAINING_DIR = "/content/drive/MyDrive/Dataset/DatasetV2/Dataset/Split_300/Training"
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation Data Path and Image gen (Normalization)
VALIDATION_DIR = "/content/drive/MyDrive/Dataset/DatasetV2/Dataset/Split_300/Validation"
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=batch_size
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=batch_size
)

Found 3120 images belonging to 13 classes.
Found 780 images belonging to 13 classes.


In [ ]:
# Just making sure the library to import things from G Drive Is present
from google.colab import drive
import os # Library to manipulate OS  I.E Copying, changing, and or reading things on folders

directoryTrain = '/content/drive/MyDrive/Dataset/DatasetV2/Dataset/Training' # Change directory depending on your file's path
directoryVal = '/content/drive/MyDrive/Dataset/DatasetV2/Dataset/Validation'

# List the contents of the directory
# Code to Check wether there is file's in the directory or not (Might be useful to debug)
try:
    files = os.listdir(directoryTrain)
    print("Files in directory:", files)
except FileNotFoundError:
    print(f"The directory {directoryTrain} does not exist.")

try:
    files = os.listdir(directoryVal)
    print("Files in directory:", files)
except FileNotFoundError:
    print(f"The directory {directoryVal} does not exist.")

Files in directory: ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Abstract']
Files in directory: ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Abstract']


In [ ]:
# Load training data
#train_data = tf.keras.utils.image_dataset_from_directory(
  #"/content/drive/MyDrive/Dataset/DatasetV2/WikiArt Art Movements Styles/Sampled_Arts_1000",
  #image_size=(224, 224),
  #batch_size=batch_size,
  #seed=123
#)

# Load validation data
#val_data = tf.keras.utils.image_dataset_from_directory(
  #"/content/drive/MyDrive/Dataset/DatasetV2/WikiArt Art Movements Styles/Sampled_Arts_200",
  #image_size=(224, 224),
  #batch_size=batch_size,
  #seed=123
#)

Found 12999 files belonging to 13 classes.
Found 2600 files belonging to 13 classes.


#Model 1


In [ ]:
# Load the Resnet50V2 pretrained model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = True

94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
# Add extra layers to model
output = base_model.output

output = Flatten()(output)

output = Dense(256, kernel_initializer='he_uniform')(output)
output = Dropout(0.5)(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)

output = Dense(128, kernel_initializer='he_uniform')(output)
output = Dropout(0.25)(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)

output = Dense(64, kernel_initializer='he_uniform')(output)
output = Dropout(0.25)(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)

output = Dense(32, kernel_initializer='he_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)

output = Dense(13, activation='softmax')(output)

model = Model(inputs=base_model.inputs, outputs=output)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                            

In [ ]:
adam = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer= adam, metrics=['accuracy'])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
)

Epoch 1/15
97/97 [==============================] - 1408s 14s/step - loss: 2.6342 - accuracy: 0.1120 - val_loss: 88390.5625 - val_accuracy: 0.0651
Epoch 2/15
97/97 [==============================] - 165s 2s/step - loss: 2.5610 - accuracy: 0.1120 - val_loss: 23.9687 - val_accuracy: 0.0781
Epoch 3/15
97/97 [==============================] - 176s 2s/step - loss: 2.5094 - accuracy: 0.1298 - val_loss: 41.9509 - val_accuracy: 0.1146
Epoch 4/15
97/97 [==============================] - 155s 2s/step - loss: 2.4629 - accuracy: 0.1331 - val_loss: 6.5987 - val_accuracy: 0.1758
Epoch 5/15
97/97 [==============================] - 154s 2s/step - loss: 2.4206 - accuracy: 0.1584 - val_loss: 2.4343 - val_accuracy: 0.1628
Epoch 6/15
97/97 [==============================] - 172s 2s/step - loss: 2.4343 - accuracy: 0.1535 - val_loss: 21.2517 - val_accuracy: 0.1042
Epoch 7/15
97/97 [==============================] - 171s 2s/step - loss: 2.3966 - accuracy: 0.1528 - val_loss: 35.2068 - val_accuracy: 0.1641
Epo

In [ ]:
# Use MobileNetV2 as the base model
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
#Define the model
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(13, activation='softmax')  # Final output layer with 5 classes
])

In [ ]:
#Compile The Model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.001),
    metrics=['accuracy']
)

#Custom Callbacks

In [ ]:
#class AccuracyThresholdCallback(tf.keras.callbacks.Callback):
    #def on_epoch_end(self, epoch, logs={}):
        #if (logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.95):
            #print('\nReached 95% accuracy for both training and validation. Stopping training.')
            #self.model.stop_training = True

#callback = AccuracyThresholdCallback()

#Training Models

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=1000,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[callback]
)

Epoch 1/1000
107/107 [==============================] - 1971s 18s/step - loss: 1.3254 - accuracy: 0.5278 - val_loss: 1.0278 - val_accuracy: 0.6214
Epoch 2/1000
107/107 [==============================] - 179s 2s/step - loss: 0.9451 - accuracy: 0.6667 - val_loss: 0.9127 - val_accuracy: 0.6743
Epoch 3/1000
107/107 [==============================] - 172s 2s/step - loss: 0.8305 - accuracy: 0.6989 - val_loss: 0.8656 - val_accuracy: 0.6851
Epoch 4/1000
107/107 [==============================] - 170s 2s/step - loss: 0.7500 - accuracy: 0.7291 - val_loss: 0.8307 - val_accuracy: 0.7043
Epoch 5/1000
107/107 [==============================] - 169s 2s/step - loss: 0.6901 - accuracy: 0.7525 - val_loss: 0.8320 - val_accuracy: 0.6947
Epoch 6/1000
107/107 [==============================] - 170s 2s/step - loss: 0.6704 - accuracy: 0.7525 - val_loss: 0.8533 - val_accuracy: 0.7031
Epoch 7/1000
107/107 [==============================] - 168s 2s/step - loss: 0.6111 - accuracy: 0.7718 - val_loss: 0.8390 - val_

In [ ]:
# Save the model
model.save('model4_ArtNaon.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# You can also use this cell as a shortcut for downloading your model
from google.colab import files
files.download("model4_ArtNaon.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>